This notebook will explore use of a simple BERT tokenizer and a simple search using similarity scores to create groups of four groups of four words from the puzzles. This will be "zero-shot" in the sense that the BERT model will have no supervision.

# Imports #

In [1]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer


# Preprocessing #
Load Connections dataset

In [2]:
df_test = pd.read_csv("/kaggle/input/the-new-york-times-connections/Connections_Data.csv")
df_test.columns
df_test = df_test.drop(['Starting Row', 'Starting Column'], axis=1) #Unecessary info


## Imputation ##

In [3]:
df_test.isna().sum()

Game ID        0
Puzzle Date    0
Word           2
Group Name     0
Group Level    0
dtype: int64

There seems to be two missing entries from the data. I went to check the actual puzzles from those days to figure out what the missing entries should be and manually entered them. It seems like "NA" from puzzle #62 mistaken got labeled as NaN.

In [4]:
df_test[df_test.Word.isna()]

,Game ID,Puzzle Date,Word,Group Name,Group Level
930,59,2023-08-09,NaN,UNSPECIFIED QUANTITIES,0
978,62,2023-08-12,NaN,PERIODIC TABLE SYMBOLS,3


In [5]:
df_test[(df_test['Game ID'] == 59) & 
    (df_test['Group Name'] == 'UNSPECIFIED QUANTITIES')]

,Game ID,Puzzle Date,Word,Group Name,Group Level
930,59,2023-08-09,NaN,UNSPECIFIED QUANTITIES,0
932,59,2023-08-09,FEW,UNSPECIFIED QUANTITIES,0
937,59,2023-08-09,HANDFUL,UNSPECIFIED QUANTITIES,0
943,59,2023-08-09,SEVERAL,UNSPECIFIED QUANTITIES,0


In [6]:
df_test.at[930, "Word"] = "SOME"

In [7]:
df_test[(df_test['Game ID'] == 62) & 
    (df_test['Group Name'] == 'PERIODIC TABLE SYMBOLS')]

,Game ID,Puzzle Date,Word,Group Name,Group Level
978,62,2023-08-12,NaN,PERIODIC TABLE SYMBOLS,3
981,62,2023-08-12,NI,PERIODIC TABLE SYMBOLS,3
983,62,2023-08-12,HE,PERIODIC TABLE SYMBOLS,3
987,62,2023-08-12,FE,PERIODIC TABLE SYMBOLS,3


In [8]:
df_test.at[978, "Word"] = "NA"

In [9]:
df_test.isna().sum()

Game ID        0
Puzzle Date    0
Word           0
Group Name     0
Group Level    0
dtype: int64